# `__init__`

In [1]:
import geopandas as gpd
import xarray as xr
import pint_xarray
import pint_pandas
import pint

from summaflow import (
    GeoLayer,
    SUMMAWorkflow,
)

import os

In [2]:
# paths
root_path_layers = '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/test-stats/'

# layers' paths
landcover_path = os.path.join(root_path_layers, 'landsat-landcover')
soilclass_path = os.path.join(root_path_layers, 'usda-soil')
merithdyr_path = os.path.join(root_path_layers, 'merit-hydro')

# geolayer's path
root_path_geoms = '../../../examples/bow-at-calgary-geofabric/'
riv_path = os.path.join(root_path_geoms, 'bcalgary_rivers.shp')
cat_path = os.path.join(root_path_geoms, 'bcalgary_subbasins.shp')
hru_path = os.path.join(root_path_geoms, 'bcalgary_subbasins.shp')

In [3]:
riv_obj = gpd.read_file(riv_path)
cat_obj = gpd.read_file(cat_path)
hru_obj = gpd.read_file(hru_path)

In [4]:
# layers needed by the setup workflow
# elevation
elv = GeoLayer.from_maf(
    maf_stats=os.path.join(merithdyr_path, 'summaflow_stats_elv.csv'),
    maf_layer=os.path.join(merithdyr_path, 'summaflow_elv.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'meters',
)
# landcover
landcover = GeoLayer.from_maf(
    maf_stats=os.path.join(landcover_path, 'summaflow_stats_NA_NALCMS_landcover_2020_30m.csv'),
    maf_layer=os.path.join(landcover_path, 'summaflow_NA_NALCMS_landcover_2020_30m.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'dimensionless',
)
# USDA soil classes
soil = GeoLayer.from_maf(
    maf_stats=os.path.join(soilclass_path, 'summaflow_stats_soil_classes.csv'),
    maf_layer=os.path.join(soilclass_path, 'summaflow_soil_classes.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'dimensionless',
)

# dummy layers for `tan_slope`, `contourLength` and `downHRUindex`
# until relevant workflows are implemented inside `gistool`
slope = elv
contour = elv
hru_index = elv

In [5]:
exp = SUMMAWorkflow(
    forcing_data = ['1', '2'],
    forcing_name_mapping = {
        'pr': 'precipitation',
        'temp': 'airtemp',
    },
    forcing_unit_mapping = {
        'pr': 'mm',
        'temp': 'degC',
    },
    forcing_to_unit_mapping = {
        'pr': 'kgm^/s',
        'temp': 'K',
    },
    forcing_attrs = {
        'measurement_height': 40,
        'measurement_height_unit': 'meter',
        'forcing_time_zone': 'utc', # original timezone of the forcing datatset
        'target_time_zone': 'utc', # if UTC, SUMMA converts to local time zone internally
        'local': {
            'pr': {
                'long_name': 'precipitation at the surface',
            },
            'temp': {
                'long_name': 'Air temperature',
            },
        },
        'global': {},
    },
    topology_data = {
        'riv': riv_obj,
        'hru': hru_obj,
        'cat': cat_obj,
    },
    topology_unit_mapping = {'a': 'm', 'c': 'ha'},
    topology_to_unit_mapping = {'a': 'bar', 'c': 'dca'},
    topology_attrs = {
        'gru_fid': 'COMID',
        'hru_fid': 'COMID',
    },
    geospatial_data= {
        'elevation': elv,
        'soilTypeIndex': soil,
        'vegTypeIndex': landcover,
        'tan_slope': slope,
        'contourLength': contour,
        'downHRUindex': hru_index,
    },
)

In [6]:
exp

Forcing files: ['1', '2']
GRUs: 169
HRUs: 169
Rivers: 169 
Initialized: False

In [7]:
exp.init_attrs()

/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/src/summaflow/utils.py:128: UserWarning: Using EPSG:3857 for centroid calculations on geographic data. For better accuracy, specify a local projection using calculation_crs parameter.
  warnings.warn(


<xarray.Dataset> Size: 22kB
Dimensions:         (gru: 169, hru: 169)
Coordinates:
  * gru             (gru) int64 1kB 71027942 71027957 ... 71039250 71039266
  * hru             (hru) int64 1kB 71027942 71027957 ... 71039250 71039266
Data variables: (12/14)
    mHeight         (hru) int64 1kB 40 40 40 40 40 40 40 ... 40 40 40 40 40 40
    slopeTypeIndex  (hru) int64 1kB 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    gruId           (gru) int64 1kB 71027942 71027957 ... 71039250 71039266
    hruId           (hru) int64 1kB 71027942 71027957 ... 71039250 71039266
    hru2gruId       (hru) int64 1kB 71027942 71027957 ... 71039250 71039266
    latitude        (hru) float64 1kB -114.2 -114.4 -114.5 ... -115.0 -115.3
    ...              ...
    tan_slope       (hru) float64 1kB 1.16e+03 1.204e+03 ... 1.913e+03 2.121e+03
    contourLength   (hru) float64 1kB 1.16e+03 1.204e+03 ... 1.913e+03 2.121e+03
    downHRUindex    (hru) float64 1kB 1.16e+03 1.204e+03 ... 1.913e+03 2.121e+03
    elevation       (hru) float64 1kB 1.16e+03 1.204e+03 ... 1.913e+03 2.121e+03
    vegTypeIndex    (hru) int64 1kB 17 10 17 17 8 8 8 8 ... 16 16 16 16 16 1 16
    soilTypeIndex   (hru) int64 1kB 3 3 3 3 3 3 3 3 3 3 ... 3 3 3 3 3 3 3 3 3 3
Attributes:
    author:   University of Calgary
    license:  GNU General Public License v3 (or any later version)
    purpose:  Created by SUMMA automated configuration scripts (SUMMAFlow) 0....

In [8]:
exp.attrs

<xarray.Dataset> Size: 22kB
Dimensions:         (gru: 169, hru: 169)
Coordinates:
  * gru             (gru) int64 1kB 71027942 71027957 ... 71039250 71039266
  * hru             (hru) int64 1kB 71027942 71027957 ... 71039250 71039266
Data variables: (12/14)
    mHeight         (hru) int64 1kB 40 40 40 40 40 40 40 ... 40 40 40 40 40 40
    slopeTypeIndex  (hru) int64 1kB 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    gruId           (gru) int64 1kB 71027942 71027957 ... 71039250 71039266
    hruId           (hru) int64 1kB 71027942 71027957 ... 71039250 71039266
    hru2gruId       (hru) int64 1kB 71027942 71027957 ... 71039250 71039266
    latitude        (hru) float64 1kB -114.2 -114.4 -114.5 ... -115.0 -115.3
    ...              ...
    tan_slope       (hru) float64 1kB 1.16e+03 1.204e+03 ... 1.913e+03 2.121e+03
    contourLength   (hru) float64 1kB 1.16e+03 1.204e+03 ... 1.913e+03 2.121e+03
    downHRUindex    (hru) float64 1kB 1.16e+03 1.204e+03 ... 1.913e+03 2.121e+03
    elevation       (hru) float64 1kB 1.16e+03 1.204e+03 ... 1.913e+03 2.121e+03
    vegTypeIndex    (hru) int64 1kB 17 10 17 17 8 8 8 8 ... 16 16 16 16 16 1 16
    soilTypeIndex   (hru) int64 1kB 3 3 3 3 3 3 3 3 3 3 ... 3 3 3 3 3 3 3 3 3 3
Attributes:
    author:   University of Calgary
    license:  GNU General Public License v3 (or any later version)
    purpose:  Created by SUMMA automated configuration scripts (SUMMAFlow) 0....